In [1]:
import os, sys
import torch
import torchvision
import torch.nn as nn

### Setting block

In [3]:
# Reference: https://github.com/andreasveit/densenet-pytorch/blob/master/densenet.py
class Basic_conv2d_LeakyReLU(nn.Module):
    def __init__(self, in_channels, out_channels, inplace, **kwarg):
        super(Basic_conv2d_LeakyReLU, self).__init__()
        
        self.conv = nn.Conv2d(in_channels=in_channels, out_channels=out_channels, **kwarg)
        self.bn = nn.BatchNorm2d(num_features=out_channels)
        self.leaky_relu = nn.LeakyReLU(negative_slope=0.2, inplace=inplace)
        
    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.leaky_relu(out)
        
        return out

class depthwise_separable_conv(nn.Module):
    def __init__(self, in_channels, kernels_per_layer, out_channels, **kwarg):
        super(depthwise_separable_conv, self).__init__()
        
        self.depthwise_conv = nn.Conv2d(in_channels=in_channels,
                                        out_channels=kernels_per_layer * in_channels, 
                                        groups=in_channels,
                                        **kwarg)
        self.pointwise_conv = nn.Conv2d(in_channels=kernels_per_layer * in_channels,
                                        out_channels=out_channels,
                                        kernel_size=(1,1),
                                        stride=(1,1),
                                        padding=(0,0))
    def forward(self, x):
        out = self.depthwise_conv(x)
        out = self.pointwise_conv(out)
        
        return out

class Bottleneck_Block(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Bottleneck_Block, self).__init__()
        
        self.conv_1x1 = Basic_conv2d_LeakyReLU(in_channels=in_channels,
                                               out_channels=out_channels,
                                               inplace=True,
                                               kernel_size=(1,1),
                                               stride=1,
                                               padding=0)
        
        self.conv_3x3 = depthwise_separable_conv(in_channels=out_channels,
                                                 kernels_per_layer=1,
                                                 out_channels=out_channels,
                                                 kernel_size=(3,3), 
                                                 stride = (1,1),
                                                 padding = (1,1))

    def forward(self, x):
        out = self.conv_1x1(x)
        out = self.conv_3x3(out)
        
        return torch.cat((x, out), dim=1)

      
class DenseBlock(nn.Module):
    def __init__(self, in_channels, growth_rate, nb_layers):
        super(DenseBlock, self).__init__()
        
        self.dense_layers = self._make_layers(in_channels, growth_rate, nb_layers)
        
    def _make_layers(self, in_channels, growth_rate, nb_layers):
        layers = []
        for i in range(nb_layers):
            layers.append(Bottleneck_Block(in_channels + i*growth_rate, growth_rate))
        
        return nn.Sequential(*layers)
    
    def forward(self, x):
        return self.dense_layers(x)

### Testing on Mnist

In [4]:
mnist_trainset = torchvision.datasets.MNIST(root='./data',
                                            train=True,
                                            download=True,
                                            transform=torchvision.transforms.ToTensor())
mnist_loader = torch.utils.data.DataLoader(dataset=mnist_trainset, batch_size=64)
x, y = mnist_loader.__iter__().__next__()

In [5]:
model = DenseBlock(in_channels=1, growth_rate=64, nb_layers=8)
model

DenseBlock(
  (dense_layers): Sequential(
    (0): Bottleneck_Block(
      (conv_1x1): Basic_conv2d_LeakyReLU(
        (conv): Conv2d(1, 64, kernel_size=(1, 1), stride=(1, 1))
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (leaky_relu): LeakyReLU(negative_slope=0.2, inplace)
      )
      (conv_3x3): depthwise_separable_conv(
        (depthwise_conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=64)
        (pointwise_conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
      )
    )
    (1): Bottleneck_Block(
      (conv_1x1): Basic_conv2d_LeakyReLU(
        (conv): Conv2d(65, 64, kernel_size=(1, 1), stride=(1, 1))
        (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (leaky_relu): LeakyReLU(negative_slope=0.2, inplace)
      )
      (conv_3x3): depthwise_separable_conv(
        (depthwise_conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), paddin

In [6]:
out = model(x)
out.size() # 1 + 64*8

torch.Size([64, 513, 28, 28])

### Reference:
1. [DenseNet: Paper](https://arxiv.org/pdf/1608.06993.pdf)
2. [DenseNet: Pytorch Tutorial](https://github.com/andreasveit/densenet-pytorch/blob/master/densenet.py)
3. [DenseNet: Review Article-1](https://blog.csdn.net/u014380165/article/details/75142664)
4. [DenseNet: Review Article-2](https://towardsdatascience.com/understanding-and-visualizing-densenets-7f688092391a)
5. [CondenseNet: Paper](https://arxiv.org/pdf/1711.09224.pdf)